In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests
import csv

#### Web Scraping

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')
tr = soup.find_all('tr')
csv_file = open('neighbourhood_data_Toronto.csv','w')
csv_writer = csv.writer(csv_file)
for y in tr[:-4]:
    x = y.text
    x = list(filter(lambda y: y != '' , x.rstrip("\n").split("\n")))
    csv_writer.writerow(x)
csv_file.close()

In [4]:
import pandas as pd

#### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [5]:
df = pd.read_csv('neighbourhood_data_Toronto.csv')
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [6]:
df = df[df.Borough!='Not assigned']
df.tail()

,Postal Code,Borough,Neighborhood
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing Centre
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [7]:
df.loc[df["Postal Code"]=="M5A"]

,Postal Code,Borough,Neighborhood
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [8]:
df.Neighborhood[df.Neighborhood=='Not assigned'] = df.Borough

In [9]:
df[df.eq('Nan').any(1)]

,Postal Code,Borough,Neighborhood


#### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [10]:
df.shape

(103, 3)